<h2>Queremos fazer uma análise de quais partidos,
mais especificamente governos de UFs que foram mais permissivos 
em relação à degradação do meio ambiente.
Para isso, precisamos de dados do meio ambiente e de eleições.

<h3>Instalando base dos dados

In [29]:
!pip install basedosdados

You should consider upgrading via the 'd:\python\python.exe -m pip install --upgrade pip' command.


Importando ferramentas

In [30]:
!pip install plotly

You should consider upgrading via the 'd:\python\python.exe -m pip install --upgrade pip' command.


In [31]:
import basedosdados as bd
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import plotly.offline as py
import plotly.graph_objs as go

<h3>Lendo tabela de informação de candidatos eleitos

<h4> Para pegar as informações relevantes, selecionamos só os governantes de UFs que têm dados sobre sua cobertura de mata amazônia na basedosdados 
     <h4>A consulta em si está na pasta de consultas SQL com o nome 'GovernoEstado.sql'
         <h4> Aqui importamos o csv gerado pela consulta descrita acima

In [32]:
csvGov= pd.read_csv('ConsultasEmCsv/governosestados.csv')

In [33]:
csvGov

,anos,sigla_uf,sigla_partido
0,1994,AC,PPR
1,1998,AC,PT
2,2002,AC,PT
3,2006,AC,PT
4,2010,AC,PT
5,2014,AC,PT
6,2018,AC,PP
7,1994,AM,PPR
8,1998,AM,PFL
9,2002,AM,PPS


<h3>Lendo tabela de informação de cobertura do bioma amazônia por estado e ano

<h4> Pegamos as informações da área do bioma Amazônia relacionada ao estado e ao ano em que foram registradas
     <h4>A consulta em si está na pasta de consultas SQL com o nome 'AreaTotAmazonia.sql'
         <h4> Aqui importamos o csv gerado pela consulta descrita acima

In [34]:
csvCob= pd.read_csv('ConsultasEmCsv/coberturaanoestado.csv')

In [35]:
csvCob

,ano,estado_abrev,areatotal
0,1985,AC,158629.949213
1,1986,AC,158481.673831
2,1987,AC,158058.639895
3,1988,AC,157346.219951
4,1989,AC,156778.336434
...,...,...,...
310,2015,TO,6951.741695
311,2016,TO,6592.706262
312,2017,TO,6853.481358
313,2018,TO,6645.391944


<h1> Queremos saber quais partidos foram mais permissivos em relação a perda do bioma da amazônia

<h3> Relacionando as duas tabelas para obter o dataset o qual utilizaremos para operações e análises no pandas, com o objetivo de responder a pergunta acima:
   

<h4> Com as duas tabelas acima, fazemos um inner join para relacionar as mudanças dos biomas com o partido governante
     <h4>A consulta que relaciona as tabelas está na pasta de consultas SQL com o nome 'JoinGovArea.sql'
         <h4> Abaixo importamos o csv gerado pela consulta descrita acima

In [36]:
lig = pd.read_csv('ConsultasEmCsv/coberturajoinpartidos.csv')

In [37]:
lig

,ano,sigla_uf,areatotal,sigla_partido
0,1994,AC,1.547424e+05,PPR
1,1998,AC,1.522341e+05,PT
2,2002,AC,1.480173e+05,PT
3,2006,AC,1.477197e+05,PT
4,2010,AC,1.465175e+05,PT
5,2014,AC,1.456493e+05,PT
6,2018,AC,1.441178e+05,PP
7,1994,AM,1.505151e+06,PPR
8,1998,AM,1.503607e+06,PFL
9,2002,AM,1.503723e+06,PPS


<h3> Adicionando coluna na qual queremos armazenar a porcentagem de cobertura de amazônia perdida por mandato de governo

In [38]:
#Usando o pandas, adicionando uma coluna inicialmente vazia
lig["delta"] = None
lig

,ano,sigla_uf,areatotal,sigla_partido,delta
0,1994,AC,1.547424e+05,PPR,None
1,1998,AC,1.522341e+05,PT,None
2,2002,AC,1.480173e+05,PT,None
3,2006,AC,1.477197e+05,PT,None
4,2010,AC,1.465175e+05,PT,None
5,2014,AC,1.456493e+05,PT,None
6,2018,AC,1.441178e+05,PP,None
7,1994,AM,1.505151e+06,PPR,None
8,1998,AM,1.503607e+06,PFL,None
9,2002,AM,1.503723e+06,PPS,None


<h3>Função para armazenar a perda de cobertura do bioma por mandato
    <h4> Nessa função, percorremos as linhas do database e pegamos a informação da área antes e após o mandato, para armazenar a porcentagem de quanto mudou na coluna delta

In [47]:
lig["delta"] = None
for index in lig.index:
    line = lig.iloc[index]
    ano = line.ano
    est = line.sigla_uf
    areaantiga = line.areatotal
    areaatual = lig[(lig.sigla_uf==est) & (lig.ano == ano + 4)].areatotal
    if len(areaatual) == 0:
        areaatual= None
    else:
        areaatual= areaatual.iloc[0]
        lig.loc[index,'delta'] = ((areaatual - areaantiga)/ areaantiga)*100

#variável 'deltatb' agora armazena os deltas de áreas
deltadb = lig
#removemos linhas em que nao temos informações do desmatamento
deltadb = deltadb.dropna()

<h4> Com a porcentagem da área perdida relacionada com o partido que estava no mandato, podemos fazer cálculos para comparar quem foi mais permissivo em relação à perda da biodiversidade

In [54]:
#Vamos usar as duas colunas, a que indica o partido que governou, e o saldo do mandato em relação a área do bioma
ptbioma= deltadb[['sigla_partido','delta'] ]
ptbioma

,sigla_partido,delta
0,PPR,-1.62098
1,PT,-2.769942
2,PT,-0.201035
3,PT,-0.813853
4,PT,-0.592542
5,PT,-1.051536
7,PPR,-0.102529
8,PFL,0.007702
9,PPS,-0.401206
10,PMDB,-0.112286


<h3> Somamos porcentagens por cada partido e fazemos uma média em relação a quantos mandatos o partido teve, para poder comparar qual foi o mais permissivo

<h4>Somando porcentagens por partido:

In [95]:
a= ptbioma.groupby('sigla_partido').delta.sum()
a

sigla_partido
PC do B    -2.601661
PDT        -4.384973
PFL       -32.003389
PMDB      -14.296561
PMN        -0.212082
PP         -0.902800
PPB        -0.822836
PPR       -18.782534
PPS       -13.117271
PROS       -0.292676
PSB        -0.126476
PSDB      -11.540079
PSL        -0.142464
PT         -6.938390
PTB        -0.453156
Name: delta, dtype: float64

<h4>Contando instâncias de partidos para fazer a média

In [101]:
b = ptbioma.groupby('sigla_partido').sigla_partido.count()
b

sigla_partido
PC do B    1
PDT        4
PFL        7
PMDB       7
PMN        1
PP         1
PPB        1
PPR        3
PPS        4
PROS       1
PSB        3
PSDB       9
PSL        1
PT         6
PTB        1
Name: sigla_partido, dtype: int64

<b> Pegando a média por partido

In [105]:
mediapercentpart = a/b
mediapercentpart.sort_values(ascending=True)

sigla_partido
PPR       -6.260845
PFL       -4.571913
PPS       -3.279318
PC do B   -2.601661
PMDB      -2.042366
PSDB      -1.282231
PT        -1.156398
PDT       -1.096243
PP        -0.902800
PPB       -0.822836
PTB       -0.453156
PROS      -0.292676
PMN       -0.212082
PSL       -0.142464
PSB       -0.042159
dtype: float64